In [186]:
import numpy as np
import pandas as pd
import ast
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity

In [187]:
movies = pd.read_csv("/content/movies.csv")

In [188]:
movies

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",10-12-2009,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",19-05-2007,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",26-10-2015,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",16-07-2012,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",07-03-2012,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",04-09-1992,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],26-12-2011,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""

In [189]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",10-12-2009,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",19-05-2007,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",26-10-2015,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",16-07-2012,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",07-03-2012,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [190]:
movies.tail()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",04-09-1992,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],26-12-2011,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""id"": 3958}...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",13-10-2013,0,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"Signed, Sealed, Delivered",7.0,6
4801,0,[],http://shanghaicalling.com/,126186,[],en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,[],"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",03-05-2012,0,98.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,A New Yorker in Shanghai,Shanghai Calling,5.7,7
4802,0,"[{""id"": 99, ""name"": ""Documentary""}]",NaN,25975,"[{""id"": 1523, ""name"": ""obsession""}, {""id"": 224...",en,My Date with Drew,Ever since the second grade when he first saw ...,1.929883,"[{""name"": ""rusty bear entertainment"", ""id"": 87...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",05-08-2005,0,90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,My Date with Drew,6.3,16


In [191]:
movies = movies.merge(credits,on ='title')

In [192]:
movies.shape

(3569, 28)

In [193]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3569 entries, 0 to 3568
Data columns (total 28 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                3569 non-null   int64  
 1   genres                3569 non-null   object 
 2   homepage              1452 non-null   object 
 3   id_x                  3569 non-null   int64  
 4   keywords              3569 non-null   object 
 5   original_language_x   3569 non-null   object 
 6   original_title        3569 non-null   object 
 7   overview_x            3569 non-null   object 
 8   popularity_x          3569 non-null   float64
 9   production_companies  3569 non-null   object 
 10  production_countries  3569 non-null   object 
 11  release_date_x        3569 non-null   object 
 12  revenue               3569 non-null   int64  
 13  runtime               3569 non-null   float64
 14  spoken_languages      3569 non-null   object 
 15  status               

In [194]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id_x', 'keywords',
       'original_language_x', 'original_title', 'overview_x', 'popularity_x',
       'production_companies', 'production_countries', 'release_date_x',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average_x', 'vote_count_x', 'id_y', 'genre',
       'original_language_y', 'overview_y', 'popularity_y', 'release_date_y',
       'vote_average_y', 'vote_count_y'],
      dtype='object')

In [195]:
movies = movies[['id_x','title','overview_x','genres','keywords','status','revenue','production_companies']]

In [196]:
movies

,id_x,title,overview_x,genres,keywords,status,revenue,production_companies
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",Released,2787965087,"[{""name"": ""Ingenious Film Partners"", ""id"": 289..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",Released,961000000,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",Released,880674609,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Released,1084939099,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",Released,284139100,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]"
...,...,...,...,...,...,...,...,...
3564,692,Pink Flamingos,Notorious Baltimore criminal and underground f...,"[{""id"": 27, ""name"": ""Horror""}, {""id"": 35, ""nam...","[{""id"": 237, ""name"": ""gay""}, {""id"": 900, ""name...",Released,6000000,"[{""name"": ""Dreamland Productions"", ""id"": 407}]"
3565,13898,The Circle,Various women struggle to function in the oppr...,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 10769, ""n...",[],Released,0,"[{""name"": ""Jafar Panahi Film Productions"", ""id..."
3566,36095,Cure,A wave of gruesome murders is sweeping Tokyo. ...,"[{""id"": 80, ""name"": ""Crime""}, {""id"": 27, ""name...","[{""id"": 233, ""name"": ""japan""}, {""id"": 549, ""na...",Released,99000,"[{""name"": ""Daiei Studios"", ""id"": 881}]"
3567,14337,Primer,Friends/fledgling entrepreneurs invent a devic...,"[{""id"": 878, ""name"": ""Science Fiction""}, {""id""...","[{""id"": 1448, ""name"": ""distrust""}, {""id"": 2101...",Released,424760,"[{""name"": ""Thinkfilm"", ""id"": 446}]"


In [197]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3569 entries, 0 to 3568
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id_x                  3569 non-null   int64 
 1   title                 3569 non-null   object
 2   overview_x            3569 non-null   object
 3   genres                3569 non-null   object
 4   keywords              3569 non-null   object
 5   status                3569 non-null   object
 6   revenue               3569 non-null   int64 
 7   production_companies  3569 non-null   object
dtypes: int64(2), object(6)
memory usage: 250.9+ KB


In [198]:
movies.isnull().sum()

id_x                    0
title                   0
overview_x              0
genres                  0
keywords                0
status                  0
revenue                 0
production_companies    0
dtype: int64

In [199]:
movies.dropna(inplace=True)

In [200]:
movies.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
3564    False
3565    False
3566    False
3567    False
3568    False
Length: 3569, dtype: bool

In [201]:
movies.duplicated().sum()

196

In [202]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [203]:
def convert(obj):
  L=[]
  for i in ast.literal_eval(obj):
    L.append(i['name'])
  return L

In [204]:
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)
movies.head()

,id_x,title,overview_x,genres,keywords,status,revenue,production_companies
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",Released,2787965087,"[{""name"": ""Ingenious Film Partners"", ""id"": 289..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...",Released,961000000,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",Released,880674609,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",Released,1084939099,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...",Released,284139100,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]"


In [205]:
def convert3(obj):
  L=[]
  counter=0
  for i in ast.literal_eval(obj):
    if counter != 3:
      L.append(i['name'])
      counter +=1
    else:
      break
    return L

In [206]:
movies['production_companies'] = movies['production_companies'].apply(convert3)
movies.head()

,id_x,title,overview_x,genres,keywords,status,revenue,production_companies
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",Released,2787965087,[Ingenious Film Partners]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...",Released,961000000,[Walt Disney Pictures]
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",Released,880674609,[Columbia Pictures]
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",Released,1084939099,[Legendary Pictures]
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...",Released,284139100,[Walt Disney Pictures]


In [207]:
movies['overview_x'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [208]:
new_movies = movies[['id_x','title','overview_x']]
new_movies

,id_x,title,overview_x
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."
...,...,...,...
3564,692,Pink Flamingos,Notorious Baltimore criminal and underground f...
3565,13898,The Circle,Various women struggle to function in the oppr...
3566,36095,Cure,A wave of gruesome murders is sweeping Tokyo. ...
3567,14337,Primer,Friends/fledgling entrepreneurs invent a devic...


In [209]:
new_movies['overview_x'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [210]:
cv = CountVectorizer(max_features=5000, stop_words='english')

In [211]:
cv.fit_transform(new_movies['overview_x']).toarray().shape

(3569, 5000)

In [212]:
vectors = cv.fit_transform(new_movies['overview_x']).toarray()

In [213]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0])

In [214]:
ps = PorterStemmer()

In [215]:
def stem(text):
  y=[]
  for i in text.split():
    y.append(ps.stem(i))
  return " ".join(y)

In [216]:
new_movies['overview_x'] = new_movies['overview_x'].apply(stem)

<ipython-input-216-f609d12b8b31>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_movies['overview_x'] = new_movies['overview_x'].apply(stem)


In [217]:
cosine_similarity(vectors)

array([[1.        , 0.        , 0.        , ..., 0.        , 0.05572782,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.07422696],
       [0.05572782, 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.07422696, 0.        ,
        1.        ]])

In [218]:
cosine_similarity(vectors).shape

(3569, 3569)

In [219]:
similarity = cosine_similarity(vectors)

In [220]:
similarity[0]

array([1.        , 0.        , 0.        , ..., 0.        , 0.05572782,
       0.        ])

In [221]:
similarity[0].shape

(3569,)

In [222]:
sorted(list(enumerate(similarity[0])),reverse = True, key = lambda x:x[1])[1:6]

[(161, 0.2672612419124244),
 (3099, 0.25677629550654774),
 (556, 0.16695677422593644),
 (2068, 0.15430334996209194),
 (660, 0.12964074471043288)]

In [223]:
def recommend(movie):
  movie_index = new_movies[new_movies['title']==movie].index[0]
  distances = similarity[movie_index]
  movies_list = sorted(list(enumerate(distances)),reverse = True, key = lambda x:x[1])[1:6]

  for i in movies_list:
    print(new_movies.iloc[i[0]].title)

In [224]:
recommend('Avatar')

Beowulf
Apollo 18
Tears of the Sun
The American
The Matrix


In [225]:
recommend('Iron Man')

Iron Man 2
Iron Man 3
Cradle 2 the Grave
Avengers: Age of Ultron
Resident Evil: Extinction


In [226]:
recommend('Avengers: Age of Ultron')

Iron Man
The Man from U.N.C.L.E.
Unstoppable
Space Cowboys
Muppets from Space
